# Generating random Tweets

This notebook uses Markov Chains models in order to generate random texts (with a maximum length equal to a Tweet).

Although it can generate reasonably accurate results, it's a completely synthetic dataset! Because of this, instead of using the generated data as training data for our NER model, we simply use it to study which text operation can improve its accuracy.

## 1. Generating Markov Chains model

Here we simply provide a series of raw text files in order to create our model - which we then save as a JSON file.

In [1]:
import markovify
import glob

max_tweet_size = 140
raw_text_files_path = '../../data/raw/markov_text_files/*.txt'
parsed_model_file_path = '../../data/parsed/markov_text_files/markov_weighted_chain.json'
raw_text_file_paths = glob.glob(raw_text_files_path)
raw_text_markov_models = []

# Read raw text files and generate Markov chain models
weights = []
for file_path in raw_text_file_paths:
    with open(file_path, encoding='utf-8') as file:
        text = file.read()
        markov_model = markovify.Text(text, state_size=4)
        raw_text_markov_models.append(markov_model)
        if 'religion' in file_path:
            weights.append(2)
        else:
            weights.append(1)
    
# Combine all generated models into a single one
markov_model = markovify.combine(raw_text_markov_models, weights)

In [2]:
# Generate 5 random sentences from the generated Markov chain model
for i in range(5):
    print(markov_model.make_short_sentence(max_tweet_size))

Dolokhov who was in the same yard as themselves and in a part of the law of that state.
Boris, too, with his friend Zhilinski, came to see the prince about twice a week.
On account of the liability to lesions of the mouth also contains the organism.
The mother smoothed the folds of her velvet dress picturesquely.
When the convention assembled in 1786, it was found that he had left it there.


In [3]:
# Save model as JSON
model_json = markov_model.chain.to_json()
with open(parsed_model_file_path, 'w') as json_file:
    json_file.write(model_json)

## 2. Improving the model

Even though we want to generate completely random Tweet-like texts, our aim is to improve the accuracy of our NER model for Country/Nationality/Religion/Currency recognition. Having said that, we will go through our model, generate a set amount of samples and use them as testing data.

In [4]:
import pandas as pd

# Define target datasets' paths
parsed_country_nationality_file = '../../data/parsed/parsed_country_nationality.csv'
parsed_currency_country_file = '../../data/parsed/parsed_currency_country.csv'
parsed_country_religion_file = '../../data/parsed/country_religion_files/parsed_country_religion.csv'
parsed_country_cities_file = '../../data/parsed/parsed_country_cities.csv'

# Load the necessary datasets
country_nationality_df = pd.read_csv(parsed_country_nationality_file, encoding='utf-8', compression='gzip', index_col=False)
currency_country_df = pd.read_csv(parsed_currency_country_file, encoding='utf-8', compression='gzip', index_col=False)
country_religion_df = pd.read_csv(parsed_country_religion_file, encoding='utf-8', compression='gzip', index_col=False)
country_cities_df = pd.read_csv(parsed_country_cities_file, encoding='utf-8', compression='gzip', index_col=False)

# Store unique sets
unique_country_common_names = country_nationality_df['Common Name'].astype(str).unique()
unique_country_official_names = country_nationality_df['Official Name'].astype(str).unique()
unique_country_nationalities = country_nationality_df['Nationality'].astype(str).unique()
unique_country_religions_name = country_religion_df['Religion'].astype(str).unique()
unique_country_rilogions_affiliation = country_religion_df['Affiliation'].astype(str).unique()
unique_currency_ids = currency_country_df['ID'].astype(str).unique()
unique_country_city_names = country_cities_df['City'].astype(str).unique()

In [5]:
import re
import random

# Define important word sets
important_word_dict = {
    'Country Names': list(map(lambda x : x.upper(), unique_country_common_names)),
    'Country Names (Official)': list(map(lambda x : x.upper(), unique_country_official_names)),
    'Country Nationalities': list(map(lambda x : x.upper(), unique_country_nationalities)),
    'Religion Names': list(map(lambda x : x.upper(), unique_country_religions_name)),
    'Religion Affiliations': list(map(lambda x : x.upper(), unique_country_rilogions_affiliation)),
    'Currencies': list(map(lambda x : x.upper(), unique_currency_ids)),
    'City Names': list(map(lambda x : x.upper(), unique_country_city_names))
}

def get_word_label(word):
    '''
    This method checks wether or not a word
    is considered to be 'important'.
    '''
    # Set regex for word parsing
    regex = re.compile('[^a-zA-Z]')
    
    for important_word_label, important_word_set in important_word_dict.items():
        comparable_word = regex.sub('', word).upper()
        if comparable_word in important_word_set:
            return important_word_label
    return None

In [6]:
def get_int_input(string):
    try:
        return int(input(string))
    except:
        return 0

In [7]:
def generate_testing_samples_dict(max_testing_samples, unnecessary_sample_retention_percent):
    # Define number of sentences to generate
    cur_testing_samples = 0

    testing_sample_dict = {}
    while len(testing_sample_dict.keys()) < max_testing_samples:
        batch_size = max_testing_samples - cur_testing_samples
        samples = [markov_model.make_short_sentence(max_tweet_size) for i in range(batch_size)]
        
        for sample in samples:
            important_words_dict = {}
            contains_important_word = False
            if sample is None:
                continue
            for word in sample.split():
                word_label = get_word_label(word)
                if word_label is not None:
                    important_words_dict[word_label] = important_words_dict.get(word_label, list()) + [word]
                    contains_important_word = True
            if (contains_important_word):
                print('------------------')
                print('\n| [New Sample]')
                print('| \tText: {}'.format(sample))
                print('| [Analysis results]')
                for label, word_list in important_words_dict.items():
                    print('| \t{}: {}'.format(label, word_list))
                print('| [Verification]')
                n_city_names = get_int_input('| \t[1/6] # City Names: ')
                n_country_names = get_int_input('| \t[2/6] # Country Names: ')
                n_country_nationalities = get_int_input('| \t[3/6] # Nationalities: ')
                n_religion_names = get_int_input('| \t[4/6] # Religion names: ')
                n_religion_affiliations = get_int_input('| \t[5/6] # Religious affiliations: ')
                n_currency_names = get_int_input('| \t[6/6] # Currency names: ')
                
                n_param_sum = n_city_names + n_country_names + n_country_nationalities + n_religion_names + n_religion_affiliations + n_currency_names
                if (n_param_sum > 0 or random.randint(0,100) <= unnecessary_sample_retention_percent):
                    testing_sample_dict[cur_testing_samples] = {
                        'Text': sample,
                        'City Names': n_city_names,
                        'Country Names': n_country_names,
                        'Country Nationalities': n_country_nationalities,
                        'Religion Names': n_religion_names,
                        'Religion Affiliations': n_religion_affiliations,
                        'Currency Names': n_currency_names
                    }
                    cur_testing_samples += 1
                    print('[Result]')
                    print('\tSAVED ({}/{})'.format(cur_testing_samples, max_testing_samples))
                else:
                    print('[Result]')
                    print('\tDISCARDED')
    return testing_sample_dict

In [8]:
def generate_auto_religion_testing_samples_dict(max_testing_samples):
    # Define number of sentences to generate
    cur_testing_samples = 0

    testing_sample_dict = {}
    while len(testing_sample_dict.keys()) < max_testing_samples:
        batch_size = max_testing_samples - cur_testing_samples
        samples = [markov_model.make_short_sentence(max_tweet_size) for i in range(batch_size)]
        
        for sample in samples:
            important_words_dict = {}
            contains_important_word = False
            if sample is None:
                continue
            for word in sample.split():
                word_label = get_word_label(word)
                if word_label in ['Religion Names', 'Religion Affiliations']:
                    important_words_dict[word_label] = important_words_dict.get(word_label, list()) + [word]
                    contains_important_word = True
            if (contains_important_word):
                print('\n---Sample----------')
                print('| [Text]')
                print('| \t{}'.format(sample))
                print('| [Analysis results]')
                for label, word_list in important_words_dict.items():
                    print('| \t{}: {}'.format(label, word_list))
                print('| [Verification]')
                
                n_religion_names = len(important_words_dict.get('Religion Names', list()))
                n_religion_affiliations = len(important_words_dict.get('Religion Affiliations', list()))
                n_param_sum = len(important_words_dict)
                if (n_param_sum > 0):
                    testing_sample_dict[cur_testing_samples] = {
                        'Text': sample,
                        'City Names': 0,
                        'Country Names': 0,
                        'Country Nationalities': 0,
                        'Religion Names': n_religion_names,
                        'Religion Affiliations': n_religion_affiliations,
                        'Currency Names': 0
                    }
                    cur_testing_samples += 1
                    print('| [Result]')
                    print('| \tSAVED ({}/{})'.format(cur_testing_samples, max_testing_samples))
                else:
                    print('| [Result]')
                    print('| \tDISCARDED')
                print('------------------')
    return testing_sample_dict

In [57]:
sample_dict = generate_testing_samples_dict(10, 10)

------------------

| [New Sample]
| 	Text: The imposing figure of Nesvitski followed by his Cossack, and the determination of the troops would not permit it.
| [Analysis results]
| 	City Names: ['of', 'of']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	DISCARDED
------------------

| [New Sample]
| 	Text: This bursts and exposes the papillae of the skin, which is almost devoid of hairs, is glistening and tightly stretched over them.
| [Analysis results]
| 	City Names: ['of', 'is', 'of', 'is']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	DISCARDED
------------------

| [New Sample]
| 	Text: This legal system combined the Teutonic civil law tradition of the southern and eastern regions of 

| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	SAVED (4/10)
------------------

| [New Sample]
| 	Text: They therefore extended the system of rewarding party workers from the public treasury.
| [Analysis results]
| 	City Names: ['of']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	DISCARDED
------------------

| [New Sample]
| 	Text: In the sphere of politics, the period witnessed the recovery of white supremacy in this way was quickly felt in national councils.
| [Analysis results]
| 	City Names: ['of', 'of']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	DISCARDED
------------------

| [New Sample]
| 	Text: For further details the read

| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	SAVED (8/10)
------------------

| [New Sample]
| 	Text: Despite the continuing reforms, ultimate authority remains in the hands of a strong foe-- But you are saying in your heart, `No foreboding!
| [Analysis results]
| 	City Names: ['of', 'a', 'are']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency names: 
[Result]
	DISCARDED
------------------

| [New Sample]
| 	Text: The founders of the American colonies and nothing seemed more reasonable to them than the accumulated winter of both poles.
| [Analysis results]
| 	City Names: ['of', 'of']
| 	Country Nationalities: ['American']
| [Verification]
| 	[1/6] # City Names: 
| 	[2/6] # Country Names: 
| 	[3/6] # Nationalities: 1
| 	[4/6] # Religion names: 
| 	[5/6] # Religious affiliations: 
| 	[6/6] # Currency nam

In [58]:
sample_df = pd.DataFrame.from_dict(sample_dict, orient='index')
sample_df.head()

,Text,City Names,Country Names,Country Nationalities,Religion Names,Religion Affiliations,Currency Names
0,"Under the Geneva Accords of 1954, Vietnam was ...",1,3,0,0,1,0
1,For decades Andorra enjoyed its status as a st...,0,1,0,0,0,0
2,"The Empress Marya, concerned for the welfare o...",0,1,0,0,0,0
3,To this the answer was made that the United St...,0,1,0,0,0,0
4,Skirmishing with the Dutch in the 17th century.,0,0,1,0,0,0


In [59]:
# Define file path for output
test_samples_file = '../../data/parsed/markov_text_files/test_samples_generic_14.csv'

sample_df.to_csv(test_samples_file, encoding='utf-8', index=False, compression='gzip')

In [60]:
# Generate a global dataframe for all the samples
all_files = glob.glob('../../data/parsed/markov_text_files/test_samples_generic_*.csv')
full_sample_df = pd.DataFrame()

for file in all_files:
    temp_df = pd.read_csv(file, encoding='utf-8', compression='gzip')
    full_sample_df = full_sample_df.append(temp_df)
    
print('Results obtained on {} tweets'.format(len(full_sample_df)))


    
# Verify the amount of values in total we have obtained for each category
sum_sample_df = pd.DataFrame(full_sample_df.sum(axis=0))
sum_sample_df.drop('Text', inplace=True)
sum_sample_df.rename(columns={0:'Number of values'}, inplace=True)

sum_sample_df

Results obtained on 150 tweets


,Number of values
City Names,18
Country Names,83
Country Nationalities,41
Religion Names,1
Religion Affiliations,2
Currency Names,3
